# Example notebook for parameter tuning

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import numba
import squidpy as sq
from numba import jit, njit, prange

'''library to train STHD model. Example codes:'''
import pandas as pd
import matplotlib.pyplot as plt
import squidpy as sq
import sys
import os
from collections import Counter

from STHD import train, sthdio, qcmask

# FUNCTION

In [23]:
## Tuning with patches

def parameter_tuning(beta, patch_path, refile, step_size = 1, max_iter = 500):
    
    sthdata = train.load_data(patch_path)
    sthdata.adata = qcmask.background_detector(sthdata.adata, threshold = 51, n_neighs =4, n_rings = 2)
    sthdata_filtered = qcmask.filter_background(sthdata, threshold = 51 )
    print(sthdata.adata.shape)
    sthdata_filtered, genemeanpd_filtered = train.sthdata_match_refgene(sthdata_filtered, refile)
    
    P_filtered, metrics = train.train(sthdata_filtered, max_iter, step_size, beta, debug=True, early_stop=True)
    P = train.fill_p_filtered_to_p_full(P_filtered, sthdata_filtered, genemeanpd_filtered, sthdata)
    sthdata = train.predict(sthdata, P, genemeanpd_filtered, mapcut= 0.8)
    pdata = train.save_prediction_pdata(sthdata, file_path = patch_path, prefix = f'beta_{beta}_stepsize_{step_size}')
    metrics = pd.DataFrame(metrics, columns=['ll','ce'])
    metrics.to_csv(os.path.join(patch_path, f'metric_beta_{beta}_stepsize_{step_size}.tsv'), sep = '\t')

def crop_patch(full_data, x1,y1,d,save_loc, sample_name = 'Visium_HD_Human_Colon_Cancer'):
    x2 = x1+d
    y2 = y1+d 
    crop_data = full_data.crop(
        x1, x2, y1, y2, 
        full_data.adata.uns['spatial'][sample_name]['scalefactors']['tissue_hires_scalef']
    )
    crop_data.save(save_loc)

def check_res(res_dir, beta_list, step_size):
    res_metrics = dict()
    for beta in beta_list:
        res_metrics[beta] = pd.read_csv(os.path.join(res_dir, f'metric_beta_{beta}_stepsize_{step_size}.tsv'), sep='\t', index_col=0)

    ref_ll = res_metrics[0].iloc[-1]['ll']
    ref_ce = res_metrics[0].iloc[-1]['ce']
    for beta in beta_list:
        cur_metric = res_metrics[beta]
        print(
            beta, '\t', 
            len(cur_metric), '\t', 
            f"{cur_metric.iloc[-1]['ll']:.2f}", '\t', 
            f"{cur_metric.iloc[-1]['ce']:.2f}", '\t',
            f"{(-cur_metric.iloc[-1]['ll'] + ref_ll) / (-ref_ll):.3f}", '\t',
            f"{(cur_metric.iloc[-1]['ce']) / ref_ce:.3f}", '\t',
        )
        plt.scatter(-cur_metric.iloc[-1]['ll'], cur_metric.iloc[-1]['ce'], label = beta)
    plt.xlabel('negative log likelihood')
    plt.ylabel('cross entropy')
    plt.legend()
    plt.yscale('log')

## Gather tuning results

def summary_single_sample(paratune_folder, beta_list, step_size):
    """ Find the best beta value for a given sample.
    Criteria: The beta that let cross entropy reduces to 10% of its original value when beta=0.
    """
    res_metrics = dict()
    for beta in beta_list:
        res_metrics[beta] = pd.read_csv(os.path.join(paratune_folder, f'metric_beta_{beta}_stepsize_{step_size}.tsv'), sep='\t', index_col=0)

    ref_ll = res_metrics[0].iloc[-1]['ll']
    ref_ce = res_metrics[0].iloc[-1]['ce']
    res = []
    for beta in beta_list:
        cur_metric = res_metrics[beta]
        res.append([
            beta, 
            len(cur_metric), 
            cur_metric.iloc[-1]['ll'],
            cur_metric.iloc[-1]['ce'],
            (-cur_metric.iloc[-1]['ll'] + ref_ll) / (-ref_ll),
            (cur_metric.iloc[-1]['ce']) / ref_ce,
            ])
    res = pd.DataFrame(res, columns=['beta', 'step', 'll', 'ce', 'll_degrade_ratio', 'ce_improve_ratio'])

    res['diff_10_ce_improve'] = np.abs(res['ce_improve_ratio'] - 0.1)

    res = res.sort_values('diff_10_ce_improve')

    best_beta = res.iloc[0]['beta']
    best_step = res.iloc[0]['step']
    
    return res, best_beta, best_step

def find_params(paratune_folder, beta_list, step_size = 1):
    """ Summarize the best beta from different samples.
    """
    patch_names = os.listdir(paratune_folder)
    res = []
    for patch in patch_names:
        _, best_beta, best_step = summary_single_sample(os.path.join(paratune_folder, patch), beta_list, step_size)
        res.append([best_beta, best_step])
    res = pd.DataFrame(res, columns=['beta', 'n_iter'])

    beta_mode = Counter(res['beta'].values).most_common()
    best_beta = beta_mode[0][0]
    best_n_iter = res[res['beta']==beta_mode[0][0]]['n_iter'].mean()
    
    print(f'Bett value that most samples find to be best: {best_beta}')
    print(f'How many samples find this beta to be the best: {100*beta_mode[0][1] / len(patch_names):.1f}%' )
    print(f"Averaged n_iter under this beta: {best_n_iter:.2f}")
    return best_beta, best_n_iter, res

# Config
- beta_list. recommended e.g. beta_list = [0, 0.1, 0.2]. Beta controls the tradeoff between optimizing loglike hood vs optimizing cross entropy. When tuning beta, we keep step_size fixed. We will check the impact of step_size in the next section.
- full_data_path contains ./square_002um, filtered_feature_bc_matrix.h5, and full resolution image
- paratune_folder for saving parameter tuning results on selected patches
- refile is the reference scRNA file generated in s01
- patches need to be selected from sample to cover regions to tune parameters on

In [25]:
full_data_path = '/hpc/group/yizhanglab/yz922/DATA/spatial/10x_HD_human_colon_cancer_20240325'

paratune_folder = './parameter_tuning' 
refile = '/hpc/group/yizhanglab/yz922/proj/STHDdev/STHD_data/colon_cancer_celltype_average_expr_genenorm_rctd_style_0430_4575gs.txt'


#beta_list = [0, 0.03, 0.1, 0.3, 1 ] # must contain 0
beta_list = [0, 0.1, 0.3]

# Manually select 6 patches for parameter tuning.
# Ideally they should be representative.
patches = {
    'crop0_left1_enteroendocrine_tumor':{'x1': 52500,'y1': 9250,'d': 1100},
    'crop1_left2_enteroendocrine_tumor':{'x1': 53450,'y1': 8360,'d': 1100},
    'crop2_gland1':{'x1': 55200,'y1': 7960,'d': 1100},
    'crop10_tumorcrypt_plasma':{'x1': 56250,'y1': 8250,'d': 1100},
    'crop5_trich':{'x1': 45450,'y1': 4100,'d': 1100},
    'crop15_tcells_with_DC':{'x1': 62800,'y1': 11000,'d': 1100}
}

# Only for first time tuning data preparation: selected represented regions on your data - loading takes time. 

In [7]:
# Load full data
full_data = sthdio.STHD(
    spatial_path = full_data_path + '/square_002um/', 
    counts_data = 'filtered_feature_bc_matrix.h5', 
    full_res_image_path= full_data_path + '/Visium_HD_Human_Colon_Cancer_tissue_image.btf', 
    load_type = 'original'
)

for patch_name in patches:
    cur_patch = patches[patch_name]
    crop_patch(full_data, cur_patch['x1'], cur_patch['y1'], cur_patch['d'], os.path.join(paratune_folder, patch_name))

/hpc/group/yizhanglab/yiz/venv/sthd_rebuttal/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/hpc/group/yizhanglab/yiz/venv/sthd_rebuttal/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


Input Pixel is aligned to 52500, 9244, 53590, 10334
creating new folder to save cropped data:  ./parameter_tuning_new_stop/crop0_left1_enteroendocrine_tumor


/hpc/group/yizhanglab/yiz/venv/sthd_rebuttal/lib/python3.8/site-packages/anndata/_core/anndata.py:1230: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c
/hpc/group/yizhanglab/yiz/venv/sthd_rebuttal/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/hpc/group/yizhanglab/yiz/venv/sthd_rebuttal/lib/python3.8/site-packages/anndata/_core/anndata.py:1230: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c
/hpc/group/yizhanglab/yiz/venv/sthd_rebuttal/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


Input Pixel is aligned to 53441, 8353, 54531, 9443
creating new folder to save cropped data:  ./parameter_tuning_new_stop/crop1_left2_enteroendocrine_tumor


/hpc/group/yizhanglab/yiz/venv/sthd_rebuttal/lib/python3.8/site-packages/anndata/_core/anndata.py:1230: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c
/hpc/group/yizhanglab/yiz/venv/sthd_rebuttal/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/hpc/group/yizhanglab/yiz/venv/sthd_rebuttal/lib/python3.8/site-packages/anndata/_core/anndata.py:1230: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c
/hpc/group/yizhanglab/yiz/venv/sthd_rebuttal/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


...

/hpc/group/yizhanglab/yiz/venv/sthd_rebuttal/lib/python3.8/site-packages/anndata/_core/anndata.py:1230: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c
/hpc/group/yizhanglab/yiz/venv/sthd_rebuttal/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/hpc/group/yizhanglab/yiz/venv/sthd_rebuttal/lib/python3.8/site-packages/anndata/_core/anndata.py:1230: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c
/hpc/group/yizhanglab/yiz/venv/sthd_rebuttal/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


Input Pixel is aligned to 45439, 4089, 46529, 5179
creating new folder to save cropped data:  ./parameter_tuning_new_stop/crop5_trich


/hpc/group/yizhanglab/yiz/venv/sthd_rebuttal/lib/python3.8/site-packages/anndata/_core/anndata.py:1230: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c
/hpc/group/yizhanglab/yiz/venv/sthd_rebuttal/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/hpc/group/yizhanglab/yiz/venv/sthd_rebuttal/lib/python3.8/site-packages/anndata/_core/anndata.py:1230: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c
/hpc/group/yizhanglab/yiz/venv/sthd_rebuttal/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


Input Pixel is aligned to 62797, 11000, 63887, 12090
creating new folder to save cropped data:  ./parameter_tuning_new_stop/crop15_tcells_with_DC


/hpc/group/yizhanglab/yiz/venv/sthd_rebuttal/lib/python3.8/site-packages/anndata/_core/anndata.py:1230: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c
/hpc/group/yizhanglab/yiz/venv/sthd_rebuttal/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/hpc/group/yizhanglab/yiz/venv/sthd_rebuttal/lib/python3.8/site-packages/anndata/_core/anndata.py:1230: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c
/hpc/group/yizhanglab/yiz/venv/sthd_rebuttal/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


# Parameter tuning on selected represented regions- training takes time. 

In [ ]:
for patch_name in patches:
    for beta in beta_list:
        print('[log] ------- Current Patch: ', patch_name, beta)
        parameter_tuning(beta, os.path.join(paratune_folder, patch_name), refile)

[log] ------- Current Patch:  crop0_left1_enteroendocrine_tumor 0


/hpc/group/yizhanglab/yiz/venv/sthd_rebuttal/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


[log] Number of spots:  22335
[Log] filtering background: 22335 spots to 21183 spots
(22335, 18085)


/hpc/group/yizhanglab/yiz/venv/sthd_rebuttal/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


[Log] num of gene overlap 4171
[Log] attaching ref gene expr in .lambda_cell_type_by_gene_matrix
cut 18085 genes to match to reference 4171 genes
[Log] prepare_constants and training weights
Currently we only support symmetric adjacency matrix of neighbors
[Log] Training...
0 102.88334040014644 -102.88334040014644 54.33869146536226
1 97.67319328211515 -97.67319328211515 55.59135570443588
2 94.18994372405021 -94.18994372405021 56.79691736187852
3 92.1248247358366 -92.1248247358366 58.79467295232333
4 90.7802418024446 -90.7802418024446 61.45132934387135
5 89.78856833039657 -89.78856833039657 64.66001265950679
6 89.04085462239007 -89.04085462239007 68.41701833620279
7 88.45725961708297 -88.45725961708297 72.7460514655003
8 87.96931428104467 -87.96931428104467 77.56729962661348
9 87.54084831238711 -87.54084831238711 82.73480168281696
10 87.16263713979541 -87.16263713979541 88.2067748805965
11 86.84753042498919 -86.84753042498919 94.14753729631438
12 86.61248312220145 -86.61248312220145 100

...

[Log] prediction saved to ./parameter_tuning_new_stop/crop0_left1_enteroendocrine_tumor/beta_0.3_stepsize_1_pdata.tsv
[log] ------- Current Patch:  crop0_left1_enteroendocrine_tumor 1


/hpc/group/yizhanglab/yiz/venv/sthd_rebuttal/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


[log] Number of spots:  22335
[Log] filtering background: 22335 spots to 21183 spots
(22335, 18085)


/hpc/group/yizhanglab/yiz/venv/sthd_rebuttal/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


[Log] num of gene overlap 4171
[Log] attaching ref gene expr in .lambda_cell_type_by_gene_matrix
cut 18085 genes to match to reference 4171 genes
[Log] prepare_constants and training weights
Currently we only support symmetric adjacency matrix of neighbors


# Find Best Parameter

In [26]:
best_beta, best_n_iter, res = find_params(paratune_folder, beta_list)

Bett value that most samples find to be best: 0.1
How many samples find this beta to be the best: 100.0%
Averaged n_iter under this beta: 23.17


In [27]:
print( best_beta, best_n_iter )

0.1 23.166666666666668
